In [1]:
# Import dependencies
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import numpy as np

In [2]:
# Create a list for each data field on a Glassdoor review
review_title = []
employee_tenure = []
reviewer_profile = []
overall_rating_number = []
work_life_balance = []
culture_and_values = []
diversity_and_inclusion = []
career_opportunities = []
compensation_and_benefits = []
senior_management = []
pros = []
cons = []
recommend = []
ceo_approval = []
business_outlook = []
errors = []

# The first page of the Microsfoft Glassdoor review
first_page = 'https://www.glassdoor.com/Reviews/Microsoft-Reviews-E1651.htm'

response = requests.get(first_page)

glassdoor_soup = bs(response.text, 'html.parser')

# Find the total pages for the review section
footer_text = glassdoor_soup.find('div', class_='paginationFooter').text
review_string_list = footer_text.split()
total_reviews = int(review_string_list[5].replace(',', ''))
total_pages = np.ceil((total_reviews/10)+1).astype(int)

#We know the total pages, begin the scraping process
print(f'Total Pages to be scraped: {total_pages}')
print("Beginning Scraping...")

# Begin looping through each of the URL review pages
for i in range(1, total_pages):
    
    # If we are not on the first page, then use a URL formatted for pages beyond the first
    if i != 1:
        url = 'https://www.glassdoor.com/Reviews/Microsoft-Reviews-E1651_P'+str(i)+'.htm?filter.iso3Language=eng'
    else:
        url = first_page

    # Use BeautifulSoup to get the needed HTML page
    response = requests.get(url)
     
    glassdoor_soup = bs(response.text, 'html.parser')
    glassdoor_body = glassdoor_soup.find('div', id = 'ReviewsFeed')
    
    try:
        glassdoor_reviews = glassdoor_body.find_all('li', class_ = 'noBorder empReview cf pb-0 mb-0')
    except:
        errors.append(str(i))
        
    # Start scanning through the webpage element's for the needed data points
    for review in glassdoor_reviews:
        
        # Title of the review
        review_header = review.find_all('a', class_ = 'reviewLink')
        for header in review_header:
            review_title.append(header.text)
        
        # Date the review was posted
        employee_time = review.find_all('span', class_ = 'pt-xsm pt-md-0 css-1qxtz39 eg4psks0')
        for time in employee_time:
            employee_tenure.append(time.text)
        
        # Includes position, location, and if they are still employed by the company
        review_profile = review.find_all('span', class_ = 'common__EiReviewDetailsStyle__newUiJobLine')
        for profile in review_profile:
            reviewer_profile.append(profile.text)
        
        # Overall rating left by the reviewer
        employee_rating = review.find_all('span', class_ = 'ratingNumber mr-xsm')
        for rating in employee_rating:
            overall_rating_number.append(rating.text)
        
        # Positive comments left by the reviewer
        pro_ratings = review.find_all('span', attrs={'data-test': 'pros'})
        for rating in pro_ratings:
            pros.append(rating.text)
        
        # Critical feedback of the position
        con_ratings = review.find_all('span', attrs={'data-test': 'cons'})
        for rating in con_ratings:
            cons.append(rating.text)
        
        # Navigate through the additional review details dropdown menu
        dropdown_ratings = review.find_all('div', class_='tooltipContainer')
        flat_ratings = review.find_all('div', class_='d-flex flex-column align-items-start')
        
        # If there is a dropdown menu, then proceed, else move on to using the overall rating for the dropdown ratings
        if dropdown_ratings:
            for dropdown in dropdown_ratings:
                
                toolbars = dropdown.find_all('ul', class_='pl-0')
                for bars in toolbars:
                    
                    # Set up a counter to see if ratings were posted by the reviewer
                    work_life_counter = 0
                    culture_and_values_counter = 0
                    diversity_and_inclusion_counter = 0
                    career_opportunities_counter = 0
                    compensation_and_benefits_counter = 0
                    senior_management_counter = 0
                    
                    # Find every list item within a dropdown menu if they exist
                    work_life = bars.find_all('li')
                    
                    # Loop through each existing item
                    for a, wl in enumerate(work_life):
                        
                        # CSS class tag assigned to ratings based on a 1-5 scale
                        one_star = r'css.1mfncox'
                        two_stars = r'css.1lp3h8x'
                        three_stars = r'css.k58126'
                        four_stars = r'css.94nhxw'
                        five_stars = r'css.11w4osi'
                        
                        # Scan the list item to see if "Work/Life Balance" is within the HTML element
                        if re.search('Work', str(wl)):
                            
                            # If it is a work/life balance rating, increase the counter by 1
                            work_life_counter = 1
                            
                            # Scan the HTML element for the CSS class tags listed above  
                            if re.search(one_star, str(wl)):
                                work_life_balance.append(1)
                            elif re.search(two_stars, str(wl)):
                                work_life_balance.append(2)
                            elif re.search(three_stars, str(wl)):
                                work_life_balance.append(3)
                            elif re.search(four_stars, str(wl)):
                                work_life_balance.append(4)
                            elif re.search(five_stars, str(wl)):
                                work_life_balance.append(5)
                            
                            
                        elif re.search('Culture', str(wl)):
                            culture_and_values_counter = 1
                            if re.search(one_star, str(wl)):
                                culture_and_values.append(1)
                            elif re.search(two_stars, str(wl)):
                                culture_and_values.append(2)
                            elif re.search(three_stars, str(wl)):
                                culture_and_values.append(3)
                            elif re.search(four_stars, str(wl)):
                                culture_and_values.append(4)
                            elif re.search(five_stars, str(wl)):
                                culture_and_values.append(5)
                        
                            
                        elif re.search('Diversity', str(wl)):
                            diversity_and_inclusion_counter = 1
                            if re.search(one_star, str(wl)):
                                diversity_and_inclusion.append(1)
                            elif re.search(two_stars, str(wl)):
                                diversity_and_inclusion.append(2)
                            elif re.search(three_stars, str(wl)):
                                diversity_and_inclusion.append(3)
                            elif re.search(four_stars, str(wl)):
                                diversity_and_inclusion.append(4)
                            elif re.search(five_stars, str(wl)):
                                diversity_and_inclusion.append(5)
                            
                        elif re.search('Career', str(wl)):
                            career_opportunities_counter = 1
                            if re.search(one_star, str(wl)):
                                career_opportunities.append(1)
                            elif re.search(two_stars, str(wl)):
                                career_opportunities.append(2)
                            elif re.search(three_stars, str(wl)):
                                career_opportunities.append(3)
                            elif re.search(four_stars, str(wl)):
                                career_opportunities.append(4)
                            elif re.search(five_stars, str(wl)):
                                career_opportunities.append(5)
                            
                        elif re.search('Compensation', str(wl)):
                            compensation_and_benefits_counter = 1
                            if re.search(one_star, str(wl)):
                                compensation_and_benefits.append(1)
                            elif re.search(two_stars, str(wl)):
                                compensation_and_benefits.append(2)
                            elif re.search(three_stars, str(wl)):
                                compensation_and_benefits.append(3)
                            elif re.search(four_stars, str(wl)):
                                compensation_and_benefits.append(4)
                            elif re.search(five_stars, str(wl)):
                                compensation_and_benefits.append(5)
                            
                        elif re.search('Senior', str(wl)):
                            senior_management_counter = 1
                            if re.search(one_star, str(wl)):
                                senior_management.append(1)
                            elif re.search(two_stars, str(wl)):
                                senior_management.append(2)
                            elif re.search(three_stars, str(wl)):
                                senior_management.append(3)
                            elif re.search(four_stars, str(wl)):
                                senior_management.append(4)
                            elif re.search(five_stars, str(wl)):
                                senior_management.append(5)
                        
                        # If there is an error, the title and error will be printed
                        else:
                            print(header.text)
                            print('---------Error Here ---------')
                    
                    # If the counter for a rating is equal to 0, append "N/A" for that datapoint
                    if work_life_counter == 0:
                        work_life_balance.append('N/A')
                    if culture_and_values_counter == 0:
                        culture_and_values.append('N/A')
                    if diversity_and_inclusion_counter == 0:
                        diversity_and_inclusion.append('N/A')
                    if career_opportunities_counter == 0:
                        career_opportunities.append('N/A')
                    if compensation_and_benefits_counter == 0:
                        compensation_and_benefits.append('N/A')
                    if senior_management_counter == 0:
                        senior_management.append('N/A')
        
        # If there is no dropdown menu, use the overall rating as the datapoint for each dropdown rating
        elif flat_ratings:
            
            for flat in flat_ratings:
                
                flat_rate_number = flat.find_all('span', class_ = 'ratingNumber')
                
                for num in flat_rate_number:
                    
                    work_life_balance.append(int(num.text[:1]))
                    culture_and_values.append(int(num.text[:1]))
                    diversity_and_inclusion.append(int(num.text[:1]))
                    career_opportunities.append(int(num.text[:1]))
                    compensation_and_benefits.append(int(num.text[:1]))
                    senior_management.append(int(num.text[:1]))
        
        # If there is an error, or the overall rating cannot be found, put an error for each data point.
        else:
            work_life_balance.append('Error')
            culture_and_values.append('Error')
            diversity_and_inclusion.append('Error')
            career_opportunities.append('Error')
            compensation_and_benefits.append('Error')
            senior_management.append('Error')
        
        # Find the element holding the CEO Approval, Recommend, and Business Outlook reivews
        review_mark = review.find_all('div', class_ = 'd-flex align-items-center mr-std')
        
        # Find the individual reviews
        for mark in review_mark:            
            
            fill_color = mark.find_all('span', class_ = 'SVGInline')
            
            for color in fill_color:
                
                # The CSS class assigned to the different answer choices for the ratings
                checkmark = {'class': ['SVGInline', 'css-hcqxoa', 'd-flex']}
                cross = {'class': ['SVGInline', 'css-1kiw93k', 'd-flex']}
                indifferent = {'class': ['SVGInline', 'css-1h93d4v', 'd-flex']}
                no_answer = {'class': ['SVGInline', 'css-10xv9lv', 'd-flex']}
                
                review_type = mark.find_all('span')
                
                # Distinguish in which datapoint the element belongs
                for review in review_type:
                    if review.text == 'Recommend':
                        if color.attrs == checkmark:
                            recommend.append('Yes')
                        elif color.attrs == cross:
                            recommend.append('No')
                        elif color.attrs == indifferent:
                            recommend.append('Indifferent')
                        elif color.attrs == no_answer:
                            recommend.append('No answer')
                    if review.text == 'CEO Approval':
                        if color.attrs == checkmark:
                            ceo_approval.append('Approve')
                        elif color.attrs == cross:
                            ceo_approval.append('Disapprove')
                        elif color.attrs == indifferent:
                            ceo_approval.append('Indifferent')
                        elif color.attrs == no_answer:
                            ceo_approval.append('No answer')
                    if review.text == 'Business Outlook':
                        if color.attrs == checkmark:
                            business_outlook.append('Good')
                        elif color.attrs == cross:
                            business_outlook.append('Bad')
                        elif color.attrs == indifferent:
                            business_outlook.append('Neutral')
                        elif color.attrs == no_answer:
                            business_outlook.append('No answer')
    if i % 1000 == 0:
        print(f'Page {i} is complete')
print('Scraping Complete')

Total Pages to be scraped: 3606
Beginning Scraping...
Page 1000 is complete
Page 2000 is complete
Page 3000 is complete
Scraping Complete


In [11]:
# Create a zipped list from all of the data gather thus far 
glassdoor_data = list(zip(review_title, employee_tenure,
                            reviewer_profile, overall_rating_number, work_life_balance, culture_and_values,
                            diversity_and_inclusion, career_opportunities, compensation_and_benefits,
                            senior_management, pros, cons, recommend,
                            ceo_approval, business_outlook))

# Create a dataframe from the zipped list
glassdoor_df = pd.DataFrame(glassdoor_data, columns = 
                            ['Review Title','Reviewer Position',
                             'Review Profile','Rating', 'Work-Life Balance', 'Culture and Values', 
                             'Diversity and Inclusion', 'Career opportunities', 'Compensation and Benefits',
                             'Senior Management', 'Pros','Cons',
                             'Recommend','CEO Approval','Business Outlook'])

# Have the index start at one for personal aesthetic reasons
glassdoor_df.index += 1

# Create a copy of the DataFrame for manipulation
glassdoor_manip_df = glassdoor_df.copy()

glassdoor_manip_df = glassdoor_manip_df.drop_duplicates(subset=['Pros', 'Cons'], keep='last')

glassdoor_manip_df

,Review Title,Reviewer Position,Review Profile,Rating,Work-Life Balance,Culture and Values,Diversity and Inclusion,Career opportunities,Compensation and Benefits,Senior Management,Pros,Cons,Recommend,CEO Approval,Business Outlook
1,Great company,"Current Employee, more than 3 years","Jan 24, 2023 - Senior Software Engineer in Red...",5.0,5,5,5,5,4,5,"Good salary, great work-life balance, great pe...","Too much bureaucracy, not so great engineering...",Yes,Approve,Good
2,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013 - Anonymous Employee in Redmond, WA",4.0,4,2,N/A,2,4,N/A,"1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,No answer,No answer,No answer
3,Awesome technology,Current Employee,"Jan 24, 2023 - Senior PM in Reston, VA",5.0,5,5,5,5,5,5,Awesome mission and great pay package,"Silos in the organization, isolating due to re...",No answer,No answer,No answer
6,Microsoft is good,Former Employee,"Jan 23, 2023 - Software Researcher",5.0,5,5,5,5,5,5,Good time part time researching at MS Research,"None really, I had a good boss.",No answer,No answer,No answer
9,Nice place to work at,"Current Employee, more than 3 years","Jan 21, 2023 - Senior Software Engineer in Bel...",5.0,5,5,5,4,4,4,Multiple interesting project. Collaborative en...,Too much politics and bureaucracy.,Yes,Approve,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34574,OK employer,Current Employee,"Dec 16, 2012 - Anonymous Employee",3.0,4,3,N/A,4,4,3,good company with stable income,this company is not the leader any more. no in...,No,Disapprove,Bad
34575,Good experience,"Current Employee, more than 5 years","Jan 2, 2013 - Anonymous Employee",3.0,3,2,N/A,3,3,4,Nice benefits location and pay,Corporate culture crazy review system and Too...,Yes,Indifferent,Neutral
34576,A pioneering technical company that values inf...,Former Employee,"Jan 9, 2013 - Anonymous Employee",4.0,4,5,N/A,3,3,4,Big name. Well organized. Great company culture.,In need of higher attention to visual developm...,No answer,No answer,No answer
34577,Its an honor to work here,"Current Employee, more than 1 year","Aug 27, 2013 - Regional Operations Lead in Ban...",4.0,4,4,N/A,3,1,4,Work Life Balance solely depends on you,A good place with lot of challenges for growth.,Yes,Approve,Good


In [12]:
# Split the Reviewer Position datafield into their current status and duration of employement
glassdoor_manip_df[['Employee Status', 'Duration of Employment']] = glassdoor_manip_df['Reviewer Position'].str.split(', ', 1, expand=True)

# Split the Reviewer Profile into the date the review was made and location in which they worked
glassdoor_manip_df[['Review Date', 'Reviewer Position/Location']] = glassdoor_manip_df['Review Profile'].str.split(' - ', 1, expand=True)

# Split the Reviewer Position/Location by their Role and Location
glassdoor_manip_df[['Reviewer Role', 'Reviewer Location']] = glassdoor_manip_df['Reviewer Position/Location'].str.split('in ', 1, expand=True)

# Split the Reviewer location up by City and State
glassdoor_manip_df[['City', 'State']] = glassdoor_manip_df['Reviewer Location'].str.split(', ', 1, expand=True)

#Convert the review date to a datetime datatype
glassdoor_manip_df['Review Date'] = pd.to_datetime(glassdoor_manip_df['Review Date'])

#Add the company to each of the records for future grouping reasons
glassdoor_manip_df['Company'] = 'Microsoft'

glassdoor_manip_df

C:\Users\EddieFlores\AppData\Local\Temp\ipykernel_8872\4144339398.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  glassdoor_manip_df[['Employee Status', 'Duration of Employment']] = glassdoor_manip_df['Reviewer Position'].str.split(', ', 1, expand=True)
C:\Users\EddieFlores\AppData\Local\Temp\ipykernel_8872\4144339398.py:5: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  glassdoor_manip_df[['Review Date', 'Reviewer Position/Location']] = glassdoor_manip_df['Review Profile'].str.split(' - ', 1, expand=True)
C:\Users\EddieFlores\AppData\Local\Temp\ipykernel_8872\4144339398.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  glassdoor_manip_df[['Reviewer Role', 'Reviewer Location']] = glassdoor_manip_df['Reviewer Position

,Review Title,Reviewer Position,Review Profile,Rating,Work-Life Balance,Culture and Values,Diversity and Inclusion,Career opportunities,Compensation and Benefits,Senior Management,...,Business Outlook,Employee Status,Duration of Employment,Review Date,Reviewer Position/Location,Reviewer Role,Reviewer Location,City,State,Company
1,Great company,"Current Employee, more than 3 years","Jan 24, 2023 - Senior Software Engineer in Red...",5.0,5,5,5,5,4,5,...,Good,Current Employee,more than 3 years,2023-01-24,"Senior Software Engineer in Redmond, WA",Senior Software Engineer,"Redmond, WA",Redmond,WA,Microsoft
2,Thoughts after 10 years....,"Current Employee, more than 10 years","Jan 28, 2013 - Anonymous Employee in Redmond, WA",4.0,4,2,N/A,2,4,N/A,...,No answer,Current Employee,more than 10 years,2013-01-28,"Anonymous Employee in Redmond, WA",Anonymous Employee,"Redmond, WA",Redmond,WA,Microsoft
3,Awesome technology,Current Employee,"Jan 24, 2023 - Senior PM in Reston, VA",5.0,5,5,5,5,5,5,...,No answer,Current Employee,None,2023-01-24,"Senior PM in Reston, VA",Senior PM,"Reston, VA",Reston,VA,Microsoft
6,Microsoft is good,Former Employee,"Jan 23, 2023 - Software Researcher",5.0,5,5,5,5,5,5,...,No answer,Former Employee,None,2023-01-23,Software Researcher,Software Researcher,None,None,None,Microsoft
9,Nice place to work at,"Current Employee, more than 3 years","Jan 21, 2023 - Senior Software Engineer in Bel...",5.0,5,5,5,4,4,4,...,Good,Current Employee,more than 3 years,2023-01-21,"Senior Software Engineer in Bellevue, WA",Senior Software Engineer,"Bellevue, WA",Bellevue,WA,Microsoft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34574,OK employer,Current Employee,"Dec 16, 2012 - Anonymous Employee",3.0,4,3,N/A,4,4,3,...,Bad,Current Employee,None,2012-12-16,Anonymous Employee,Anonymous Employee,None,None,None,Microsoft
34575,Good experience,"Current Employee, more than 5 years","Jan 2, 2013 - Anonymous Employee",3.0,3,2,N/A,3,3,4,...,Neutral,Current Employee,more than 5 years,2013-01-02,Anonymous Employee,Anonymous Employee,None,None,None,Microsoft
34576,A pioneering technical company that values inf...,Former Employee,"Jan 9, 2013 - Anonymous Employee",4.0,4,5,N/A,3,3,4,...,No answer,Former Employee,None,2013-01-09,Anonymous Employee,Anonymous Employee,None,None,None,Microsoft
34577,Its an honor to work here,"Current Employee, more than 1 year","Aug 27, 2013 - Regional Operations Lead in Ban...",4.0,4,4,N/A,3,1,4,...,Good,Current Employee,more than 1 year,2013-08-27,Regional Operations Lead in Bangalore,Regional Operations Lead,Bangalore,Bangalore,None,Microsoft


In [13]:
# Create an order to use newly created columns for the DataFrame
cols = ['Company', 'Review Title', 'Rating', 'Work-Life Balance', 'Culture and Values', 'Diversity and Inclusion',
        'Career opportunities', 'Compensation and Benefits',
        'Senior Management','Reviewer Role', 'Employee Status', 'City', 'State', 'Duration of Employment',
        'Review Date', 'Pros', 'Cons', 'Recommend', 'CEO Approval', 'Business Outlook']

glassdoor_manip_df = glassdoor_manip_df[cols]

# Replace any blanks with 'N/A' and strip excess whitespace from the datapoints
glassdoor_manip_df['Reviewer Role'] = glassdoor_manip_df['Reviewer Role'].replace('', 'N/A', regex=True)
glassdoor_manip_df.columns = glassdoor_manip_df.columns.str.strip()

glassdoor_manip_df

C:\Users\EddieFlores\AppData\Local\Temp\ipykernel_8872\719608860.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glassdoor_manip_df['Reviewer Role'] = glassdoor_manip_df['Reviewer Role'].replace('', 'N/A', regex=True)


,Company,Review Title,Rating,Work-Life Balance,Culture and Values,Diversity and Inclusion,Career opportunities,Compensation and Benefits,Senior Management,Reviewer Role,Employee Status,City,State,Duration of Employment,Review Date,Pros,Cons,Recommend,CEO Approval,Business Outlook
1,Microsoft,Great company,5.0,5,5,5,5,4,5,Senior Software Engineer,Current Employee,Redmond,WA,more than 3 years,2023-01-24,"Good salary, great work-life balance, great pe...","Too much bureaucracy, not so great engineering...",Yes,Approve,Good
2,Microsoft,Thoughts after 10 years....,4.0,4,2,N/A,2,4,N/A,Anonymous Employee,Current Employee,Redmond,WA,more than 10 years,2013-01-28,"1. If you love tech, this is a great place. ...",Brand on Your Resume: After many years of los...,No answer,No answer,No answer
3,Microsoft,Awesome technology,5.0,5,5,5,5,5,5,Senior PM,Current Employee,Reston,VA,None,2023-01-24,Awesome mission and great pay package,"Silos in the organization, isolating due to re...",No answer,No answer,No answer
6,Microsoft,Microsoft is good,5.0,5,5,5,5,5,5,Software Researcher,Former Employee,None,None,None,2023-01-23,Good time part time researching at MS Research,"None really, I had a good boss.",No answer,No answer,No answer
9,Microsoft,Nice place to work at,5.0,5,5,5,4,4,4,Senior Software Engineer,Current Employee,Bellevue,WA,more than 3 years,2023-01-21,Multiple interesting project. Collaborative en...,Too much politics and bureaucracy.,Yes,Approve,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34574,Microsoft,OK employer,3.0,4,3,N/A,4,4,3,Anonymous Employee,Current Employee,None,None,None,2012-12-16,good company with stable income,this company is not the leader any more. no in...,No,Disapprove,Bad
34575,Microsoft,Good experience,3.0,3,2,N/A,3,3,4,Anonymous Employee,Current Employee,None,None,more than 5 years,2013-01-02,Nice benefits location and pay,Corporate culture crazy review system and Too...,Yes,Indifferent,Neutral
34576,Microsoft,A pioneering technical company that values inf...,4.0,4,5,N/A,3,3,4,Anonymous Employee,Former Employee,None,None,None,2013-01-09,Big name. Well organized. Great company culture.,In need of higher attention to visual developm...,No answer,No answer,No answer
34577,Microsoft,Its an honor to work here,4.0,4,4,N/A,3,1,4,Regional Operations Lead,Current Employee,Bangalore,None,more than 1 year,2013-08-27,Work Life Balance solely depends on you,A good place with lot of challenges for growth.,Yes,Approve,Good


In [14]:
# Save the DataFrame into a csv file  
glassdoor_manip_df.to_csv('Company Reviews\Microsoft Reviews.csv', header=True, index=False)